In [ ]:

import trafficGen as et
import os
import sys
import json
import binascii



### multi client http traffic generator to nonstandard port at 200 cps customized with tcp close with RST, ttl as 10 

In [ ]:


sip=['10.31.0.19','10.31.0.18','10.31.0.2', '10.31.0.3', '10.31.0.4', '10.31.0.5', '10.31.0.6', '10.31.0.7', '10.31.0.8', '10.31.0.9', '10.31.0.10', '10.31.0.11', '10.31.0.12', '10.31.0.13', '10.31.0.14', '10.31.0.15', '10.31.0.16', '10.31.0.17']

dip='172.24.11.183'

traffic=et.AsyncClient(loglevel='INFO',log_disk='',log_console=False,dip=dip,sip=sip,dport=[3121], connections=1)
traffic.timer_options(estab_timeout=1,syn_timeout=1,receive_timeout=0.2,first_response_timeout=0.2, epoll_timeout=1)
requests=["""GET /ohs.doc HTTP/1.1\r\nHost: www.arnab.com\r\nConnection: keep-alive\r\nAccept: text/html,application/xhtml+xml,application/signed-exchange;v=b3,application/xml;q=0.9,*/*;q=0.8\r\nFrom: googlebot(at)googlebot.com\r\nUser-Agent: Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.96 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)\r\nAccept-Encoding: gzip,deflate,br\r\n\r\n"""]

request_nogzip={1:requests}
traffic.request_options(pipeline=False, request=request_nogzip)

optls=["s.setsockopt(socket.IPPROTO_IP, socket.IP_TTL, 10)","s.setsockopt(socket.SOL_SOCKET, socket.SO_LINGER, struct.pack('ii', 1, 0))"]
traffic.socket_options(optls)
traffic.objective(goal='cps', cps=200 ,dur=999999999)

results=traffic.generate()
if results:
    print(json.dumps(results, indent=4,ensure_ascii=False))


### TLS traffic - specify cipher , client cert, sni param at 200 ssl cps

In [ ]:


sip = ['172.24.11.181']
dip='172.24.11.183'


traffic=et.AsyncClient(loglevel='INFO',log_disk='',log_console=False,dip=dip,sip=sip,dport=[443], connections=1)
traffic.timer_options(estab_timeout=1,syn_timeout=1,receive_timeout=0.2,first_response_timeout=0.2, epoll_timeout=1)
traffic.ssl_options(ssl=True,cipher="ECDHE-RSA-AES128-GCM-SHA256", sslv='tlsv10',clientauth=True, sslkey='/home/ftp/clientkey2_4096clear.pem', sslcert='/home/ftp/clientcert4096_concat.pem')
requests=["""GET /ohs.doc HTTP/1.1\r\nHost: www.arnab.com\r\nConnection: keep-alive\r\nAccept: text/html,application/xhtml+xml,application/signed-exchange;v=b3,application/xml;q=0.9,*/*;q=0.8\r\nFrom: googlebot(at)googlebot.com\r\nUser-Agent: Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.96 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)\r\nAccept-Encoding: gzip,deflate,br\r\n\r\n"""]

request_nogzip={1:requests}

traffic.request_options(pipeline=False, request=request_nogzip)

traffic.objective(goal='cps', cps=200 ,dur=999999999)



results=traffic.generate()
if results:
    print(json.dumps(results, indent=4,ensure_ascii=False))


### Single long lived session - granual control over session timeouts by specifying estab_timeout, ,syn_timeout,receive_timeout,first_response_timeout, epoll_timeout,empty_poll_timeout

In [ ]:

sip=['38.45.55.28']
dip='10.107.246.93'
traffic=et.Client(loglevel='DEBUG',log_disk='',log_console=True,dip=dip, sip=sip, dport=[80])
traffic.timer_options(estab_timeout=300,syn_timeout=1,receive_timeout=300,first_response_timeout=300, epoll_timeout=300,empty_poll_timeout=300)


### Emulate and replay requests from packet caps

In [ ]:
def converthex(stream):
    templist=[]
    templist.append(str(binascii.unhexlify(stream)))
    return templist[0]

#### copy from pcap as hexstream

datahex='504f5354202f5f6170692f53502e5574696c69746965732e5350536f6369616c5377697463682e4973466f6c6c6f77696e6746656174757265456e61626c656420485454502f312e310d0a486f73743a20636f6c61626f72612e73616e696461642e676f622e65730d0a436f6e6e656374696f6e3a206b6565702d616c6976650d0a436f6e74656e742d4c656e6774683a20300d0a7365632d63682d75613a20224e6f742e412f4272616e64223b763d2238222c20224368726f6d69756d223b763d22313134222c20224d6963726f736f66742045646765223b763d22313134220d0a6163636570743a206170706c69636174696f6e2f6a736f6e3b6f646174613d766572626f73650d0a436f6e74656e742d547970653a206170706c69636174696f6e2f6a736f6e3b6f646174613d766572626f73650d0a782d726571756573746469676573743a20307839434535334230444542343131324144363330414334383533304431314239444437453745323433344539314532313834323937423638323838323937373233323036444339343130354433384536373033303145463745464443454638374232314646443534313845394141463038373230453031333136353045323743352c3133204a756e20323032332031353a30353a3234202d303030300d0a7365632d63682d75612d6d6f62696c653a203f300d0a557365722d4167656e743a204d6f7a696c6c612f352e30202857696e646f7773204e542031302e303b2057696e36343b2078363429204170706c655765624b69742f3533372e333620284b48544d4c2c206c696b65204765636b6f29204368726f6d652f3131342e302e302e30205361666172692f3533372e3336204564672f3131342e302e313832332e34330d0a7365632d63682d75612d706c6174666f726d3a202257696e646f7773220d0a4f726967696e3a2068747470733a2f2f636f6c61626f72612e73616e696461642e676f622e65730d0a5365632d46657463682d536974653a2073616d652d6f726967696e0d0a5365632d46657463682d4d6f64653a20636f72730d0a5365632d46657463682d446573743a20656d7074790d0a526566657265723a2068747470733a2f2f636f6c61626f72612e73616e696461642e676f622e65732f5369746550616765732f486f6d652e617370780d0a4163636570742d456e636f64696e673a20677a69702c206465666c6174652c2062720d0a4163636570742d4c616e67756167653a20656e2d55532c656e3b713d302e390d0a436f6f6b69653a205753535f46756c6c53637265656e4d6f64653d66616c73653b20466564417574683d3737752f5044393462577767646d567963326c76626a30694d5334774969426c626d4e765a476c755a7a30696458526d4c546769507a34385531412b4d434d755a6e78746557466b62575674596d5679633268706348427962335a705a47567966475668636d46755a3238734d434d755a6e78746557466b62575674596d5679633268706348427962335a705a47567966475668636d46755a3238734d544d7a4d7a45314e7a517a4d6a4d344e5441304d5455774c455a6862484e6c4c464a43637a4651644642615345566863554e6f4e6c6c4d6446465252323550555842756446686e6154684d5a487034533052704d30394557574a5056446772546c52484c7a5a7a655730304d7a52324c30526b52573570566a6c43646d687154586834654852714e566c505654683653564a615a574a745648427154334e31617a6c4d616c5a74576b46544c3078444d6a467a51554a4d4e325532635864534e54686a526c6f33536d56735658686b566d3153536a52355557387a636d353365586c4f596974735a314a6d53575279516b6c69616c4d76626b46704f55785556485270523368765a565233576a6c7555473546625756695955773063445a464e6d5268626e464b5a307471566d6b32515449314f576474536d5a7451314e73656c5131545852734c30786d556b5270546b6b76623068425a6b31435454427464464d79516b4a4b4c3268735755637651574e5a576d6b79655556686244464354466c686248513563486c494c7a677761304a5556486b774d6a5668636e4a77646a5972543073334b314a6a57466735636b38795247784d5a6b4a4c614738335a32316d533352725a557471536e6b304b79395a536b5a514e304a7765484e4853455a5056446b335a7a30394c4768306448427a4f6938765932397359574a76636d457563324675615752685a43356e623249755a584d76504339545544343d3b204665644175746853534f3d38736d483165584d4e62323353687231676e367959444552414e45447a48503462575647326b45647a394d592f3738616b7a314939646361484535673463784b6e506f4c4a6d535a5a4d6132672b75304a78383575343033706c483251683062486449635244306a557a303d0d0a0d0a'

sip=['10.107.246.198']
dip='10.107.246.132'
traffic=et.Client(loglevel='DEBUG',log_disk='',log_console=True,dip=dip, sip=sip, dport=[443])
traffic.timer_options(estab_timeout=1,syn_timeout=1,receive_timeout=0.4,first_response_timeout=0.4, empty_poll_timeout=2)
traffic.ssl_options(ssl=True,cipher="ECDHE-RSA-AES256-GCM-SHA384", sslv='tlsv10',clientauth=True, sslkey="/home/ftp/certificates/bloom/clientkey4096clear.pem", sslcert="/home/ftp/certificates/bloom/clientcert4096.pem")
request_1={1:[converthex(datahex)]}
traffic.request_options(pipeline=False, request=request_jumbo)

### Replay http2 traffic from pcap

In [ ]:
def converthex(stream):
    templist=[]
    templist.append(str(binascii.unhexlify(stream)))
    return templist[0]


#copy as hex stream from cap 

mg="505249202a20485454502f322e300d0a0d0a534d0d0a0d0a00001804000000000000030000006400050000400000020000000100047fffffff"

wind1="0000040800000000007fff0000"

head='001f4301250000000100000000004194acf624742f49e335055ec1f5b525a9721d7e95ff8204bd6087a633b1fe4761fc20b6649203a672d9f149aa34a62d9bfd56583e281d535a398b92483a0e2cbafe2381d001cf84c016de704f05a6db71c67e183b1f8753032a2f2a508d9bd9abfa5242cb40d25fa523b3519ce835b2ebfae83fbed00177fe8b52dc377df6800bbdf45abefb4005dd408719bad033d92a7f983d33fbed213c225449d6def2cbebdfa9243cb5f4f1dfa57f40841d950e3fed40e41a249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249249243f'

head2='834491608c567b123a182f9496a9224150831eaf4194acf624742f49e335055ec1f5b525a9721d7e95ff8740853d8698d57f999d29ad171862b3d891d0bd278cd4157b07d6d496a5c875fa5773a29d29ad171862b3'

head3="408722d2ac3b2a1c7fed40e41a2492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492492493f"

setting="000000040100000000"

get = "0000280105000000033fe11f820488627b691d485d3e53864188aa69d29ac4b9ec9b7a8825b650c3abb815c153032a2f2a"

sip=['10.107.246.199']
dip='10.12.1.149'
traffic=et.Client(loglevel='INFO',log_disk='',log_console=True,dip=dip, sip=sip, dport=[443])
traffic.timer_options(estab_timeout=500,syn_timeout=1,receive_timeout=5,first_response_timeout=5, epoll_timeout=500,empty_poll_timeout=500)

traffic.ssl_options(ssl=True,cipher="ECDHE-RSA-AES128-GCM-SHA256", sslv='tlsv10',sni='www.hktdc.com')

requests = [ converthex(magic),converthex(setting1),converthex(setting2),converthex(get)]

request_nogzip={1:requests}

traffic.request_options(pipeline=False, request=request_nogzip)

results=traffic.generate()
if results:
    print(json.dumps(results, indent=4,ensure_ascii=False))
